# 토픽모델링
- 비지도학습
- 하나 또는 여러가지 문서를 토픽으로 할당하는 작업
- LDA(Latent Dirichlet Allocation, 잠재 디리클레 할당

# LDA
- 자주 나타나는 단어의 그룹(토픽)을 찾는 것
- 주제와는 거리가 멀수도 있음
- 의미가 있는 성분을 찾는 것이기 때문

In [1]:
!pip install pandas numpy sklearn mglearn

  Using cached mglearn-0.1.9.tar.gz (540 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 3.3/3.3 MB 42.7 MB/s eta 0:00:00
  Created wheel for mglearn: filename=mglearn-0.1.9-py2.py3-none-any.whl size=582639 sha256=7bebe3fa7e1a23c8ffd12d2f6a39e03be0900fcbb939e1f69a06321e866e4a1f
  Stored in directory: c:\users\smhrd\appdata\local\pip\cache\wheels\87\75\37\404e66d0c4bad150f101c9a0914b11a8eccc2681559936e7f7
Successfully built mglearn


## 데이터 받아오기

In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv('data/ratings_train.txt' , delimiter = '\t')
df_test = pd.read_csv('data/ratings_test.txt' , delimiter = '\t')

## 결측치 제거

In [4]:
df_train.dropna(inplace = True)
df_test.dropna(inplace = True)

In [5]:
df_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [6]:
text_train = df_train['document']

## 토큰화

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# max_features = 가장 많이 등장하는 단어 10000개만 사용
# max_df = 15%이상의 문서에서 등장하는 단어 제거
cv = CountVectorizer(max_features = 10000 , max_df = 15)
X = cv.fit_transform(text_train)

In [9]:
# 149995문장을 10000개 단어로 표시 한 데이터를 벡터화
X

<149995x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 96120 stored elements in Compressed Sparse Row format>

## LDA

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

# n_components = 10 : 10개의 토픽 그룹을 만들어 달라!
# learning_method = 'batch' : 기본값 online, batch 느리지만 성능이 좋다

lda = LatentDirichletAllocation(n_components = 10, learning_method = 'batch',
                               max_iter = 25 , random_state = 0)

In [ ]:
document_topics = lda.fit_transform(X)